# Demo code for Elo rating example 

## NeurIPS 2023 Tutorial: Do you Prefer Learning with Preferences? 

Author: Aditya Gopalan

In [ ]:
# import libraries

import chess.pgn # to read chess PGN format files
import io
import pandas as pd


### The Elo rating update: 

Suppose players $i$ and $j$, having Bradley-Terry-Luce (BTL) preference model parameters $\theta_i$ and $\theta_j$, play a chess game. 

Let $R \in \left\{0, 1, \frac{1}{2} \right\}$ denote the outcome of the game, where $R=1$ represents a win for $i$. 

The Elo ratings of the players are updated as follows:

$$ \theta_i \leftarrow \theta_i + 10 \cdot (R - p_{ij}),$$ for $i$, and 

$$ \theta_j \leftarrow \theta_j + 10 \cdot (1-R - p_{ji})$$ for $j$.

Here, $p_{ij} \equiv p_{ij}(\theta_i, \theta_j) = \frac{10^{\theta_i}}{10^{\theta_i} + 10^{\theta_j}}$ is the win probability of player $i$, and $p_{ji} = 1-p_{ij}$ is the win probability of player $j$. 

### Connection to gradient descent: 

Ignoring the case where a draw $R=\frac{1}{2}$ is possible, the Elo update can be interpreted as single step of gradient descent for the logarithmic loss function 

$$ \ell(\theta_i, \theta_j)  = R \cdot \log \frac{1}{p_{ij}} + (1-R) \cdot \log \frac{1}{p_{ji}}, $$

with a step size of $10$. 

In [ ]:
# set up the Elo rating update

def calculate_elo_rating(rating1, rating2, result, k=10):
    """calculate win-loss probabilities for a pair of players based on the Bradley-Terry-Luce preference model"""
    exp_score1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
    exp_score2 = 1 / (1 + 10 ** ((rating1 - rating2) / 400))

    # Update ratings
    new_rating1 = rating1 + k * (result - exp_score1)
    new_rating2 = rating2 + k * (1 - result - exp_score2)

    return new_rating1, new_rating2

def process_pgn_file(pgn_file_path, initial_ratings):
    """reads games 1 by 1 from a PGN and updates players' Elo ratings"""
    player_ratings = initial_ratings.copy()
    result_map = {"1-0": (1, 0), "0-1": (0, 1), "1/2-1/2": (0.5, 0.5)}

    with open(pgn_file_path, 'r') as pgn:
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break

            white_full_name = game.headers["White"]
            black_full_name = game.headers["Black"]
            white = white_full_name.split(',')[0]
            black = black_full_name.split(',')[0]
            result = game.headers["Result"]
            pre_rating_white = player_ratings[white_full_name]
            pre_rating_black = player_ratings[black_full_name]

            # Update ratings
            new_ratings = calculate_elo_rating(
                pre_rating_white,
                pre_rating_black,
                result_map[result][0]
            )

            player_ratings[white_full_name], player_ratings[black_full_name] = new_ratings

            # Print game result and rating updates on separate lines
            game_result_str = 'draws' if result == '1/2-1/2' else 'beats'
            print(f"{white} {game_result_str} {black},\n{white}'s rating updated from {pre_rating_white} --> {player_ratings[white_full_name]},\n{black}'s rating updated from {pre_rating_black} --> {player_ratings[black_full_name]}\n")

    return player_ratings




In [ ]:
# Initial Elo ratings of the players before the tournament began

initial_ratings = {
    "Caruana, Fabiano": 2795,
    "Firouzja, Alireza": 2777,
    "Nepomniachtchi, Ian": 2771,
    "Giri, Anish": 2752,
    "So, Wesley": 2752,
    "Rapport, Richard": 2748,
    "Dominguez Perez, Leinier": 2745,
    "Vachier-Lagrave, Maxime": 2734,
    "Duda, Jan-Krzysztof": 2731,
    "Aronian, Levon": 2727
}

# Path to your chess PGN file
pgn_file_path = './sinqcup23.pgn'

In [ ]:
# Process the PGN file and compute updated Elo ratings
updated_ratings = process_pgn_file(pgn_file_path, initial_ratings)

# Create and print the final table
final_table = pd.DataFrame.from_dict({
    "Player Name": [name for name in initial_ratings.keys()],
    "Starting ELO": initial_ratings.values(),
    "Ending ELO": [int(round(updated_ratings[name], 0)) for name in initial_ratings.keys()]
})
print(final_table)
print("\n\n")